In [ ]:
import weave
from weave.monitoring import StreamTable
from weave.stream_data_interfaces import TraceSpanDict

In [ ]:
WB_ENTITY = "timssweeney"
WB_PROJECT = "trace_quickstart"
WB_STREAM = "log_stream"

st = StreamTable(f"{WB_ENTITY}/{WB_PROJECT}/{WB_STREAM}")

In [ ]:
import random
from uuid import uuid4
import datetime
import json

names = [
"fit",
"predict",
"transform",
"compile",
"evaluate",
"GridSearchCV",
"train_test_split",
"Sequential",
"early_stopping",
"OneHotEncoder",
"Dense",
"MinMaxScaler",
"dropout",
"confusion_matrix",
"lstm",
]

def simple_dict():
    return {
        "a_1": 42,
        "a_2": "Test",
        "a_3": True,
        "a_4": {
            "_kind": "AGENT",
            "list": [1,2,3],
            "dict": {
                "hello": "world"
            }
        }
    }

def create_random_span(accumulator, trace_id=None, parent_id=None, start_time_s=None):
    name = random.choice(names)
    kind = random.choice([None, 'CHAIN','AGENT','TOOL','LLM',])
    
    status_code = "UNSET"
    status = random.randint(0, 10000)
    if status == 0:
        status_code = "ERROR"
    elif status <= 9995:
        status_code = "SUCCESS"
    exception = random.choice([None, "Error"]) if status_code == "ERROR" else None
    span_id = str(uuid4())
    if trace_id is None:
        trace_id = str(uuid4())
    attributes = simple_dict()

    # Special Attributes
    attributes["span_kind"] = kind
    attributes["model"] = {
        "id": "4edfghu7654edfg",
        "obj": json.dumps(simple_dict())
    }
    inputs = simple_dict()
    output = simple_dict()
    summary = simple_dict()
    
    if start_time_s is None:
        start_time_s = datetime.datetime.now().timestamp()

    if parent_id is None:
        num_children = random.choice([1,2,3,4,5])
    else:
        num_children = random.choice([0,0,0,0,0,1,2])

    if num_children == 0 or status_code == 'ERROR':
        duration_s = random.randint(1, 10000) / 1000
    else:
        duration_s = 0
        for i in range(num_children):
            child_span = create_random_span(accumulator, trace_id, span_id, duration_s + start_time_s)
            duration_s += child_span['end_time_s'] - child_span['start_time_s']
            if child_span['status_code'] == 'ERROR':
                status_code = 'ERROR'
                exception = child_span['exception']
                break
        
    end_time_s = start_time_s + duration_s
    self_span = TraceSpanDict(
        name=name,
        span_id=span_id,
        trace_id=trace_id,
        status_code=status_code,
        start_time_s=start_time_s,
        end_time_s=end_time_s,
        parent_id=parent_id,
        attributes=attributes,
        inputs=inputs,
        output=output,
        summary=summary,
        exception=exception,
    )
    # Purely a simulation hack
    self_span["timestamp"] = datetime.datetime.fromtimestamp(start_time_s)
    accumulator.append(self_span)
    return self_span

In [ ]:
start_time_s = datetime.datetime.now().timestamp()
for i in range(10):
    acc = []
    create_random_span(acc, start_time_s=start_time_s)
    st.log(acc)
    start_time_s = max([a['timestamp'].timestamp() for a in acc])

st.finish()